In [1]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import pandas as pd
import xml.etree.ElementTree as ET
from lxml import etree
from bs4 import BeautifulSoup
import os
import json
import time
import datetime

In [2]:
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [3]:
data_dir = "../../data/siap"

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [4]:
with open("idsSIAP.json") as json_file:
    dict_ids = json.load(json_file)

In [5]:
dict_cultivos = {k.split('_')[1]:v for k,v in dict_ids.items() if k.startswith("cultivo")}
dict_ciclos = {k.split('_')[1]:v for k,v in dict_ids.items() if k.startswith("ciclo")}
dict_mes = {k.split('_')[1]:v for k,v in dict_ids.items() if k.startswith("mes")}
dict_modalidad = {k.split('_')[1]:v for k,v in dict_ids.items() if k.startswith("modalidad")}

In [6]:
url = "https://nube.siap.gob.mx/avance_agricola/"

payload = {'xajax': 'reporte', # Para obtener la tabla
'xajaxr': '1696449941927', # Timestamp UNIX
'xajaxargs[]': [
    '1', # 1: Desglose por estados, 2: Cultivo total
    '2023', # Anio
    '5', # ID Ciclo
    '3', # ID Modalidad
    '0', # ID Estado (0: Nacional)
    '--',
    '--',
    '7', # ID Cultivo
    '200201',
    '0',
    '1', # 1: Por municipio
    'undefined',
    'undefined',
    'undefined',
    '8' # Valor del mes
    ]
}

headers = {
  'Cookie': 'PHPSESSID=45ri2k73cbp2iptcrufu88p360',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

In [7]:
for year in range(2020,2024):
    payload['xajaxargs[]'][1] = str(year)
    
    for key_mes in dict_mes:
        payload['xajaxargs[]'][14] = dict_mes[key_mes]

        for key_cultivo in dict_cultivos:
            if dict_cultivos[key_cultivo] == "0":
                continue
                
            if os.path.exists(f"{data_dir}/{key_cultivo.lower().replace(' ','_')}_{key_mes.lower()}_{year}.csv"):
                continue

            payload['xajaxargs[]'][7] = dict_cultivos[key_cultivo]

            response = requests.request("POST", url, headers=headers, data=payload, verify=False)
            response.encoding='ISO-8859-1'

            #extract the cdata as a string
            root = etree.fromstring(response.content)
            cd = root.xpath('//cmd//text()')[0]

            parsed_html = BeautifulSoup(str(cd))
            parsed_table = parsed_html.body.find("table")

            if parsed_table is None:
                continue

            df_iter = pd.read_html(str(parsed_html.body.find("table")))[0]

            df_iter.to_csv(f"{data_dir}/{key_cultivo.lower().replace(' ','_')}_{key_mes.lower()}_{year}.csv",index=False)
            time.sleep(0.1)

In [8]:
with open("../../logs/info_SIAP.txt", 'w') as f:
    f.write("Archivos sobre cultivos a nivel municipal\n")
    info = f"""
    Datos de superficie cultivada, cosechada y siniestrada para diversos cultivos a nivel nacional.

    Los datos se obtuvieron del Servicio de Informacion Agroalimentaria y Pesquera de la Secretaria de Agricultura y Desarrollo Rural
    con fecha desde el 1 de enero de 2020 hasta la fecha ({datetime.datetime.today().strftime("%d-%m-%Y")})

    Los datos son recibidos en forma de tablas HTML y almacenados en archivos CSVs para su posterior tratamiento.
    """ 
    f.write(info + '\n')
    f.write("Descargado el " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
    f.write("Desde: " + url + "\n")